In [1]:
import os
import re
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    precision_score,
    cohen_kappa_score,
    classification_report,
)
import warnings

warnings.filterwarnings("ignore")

In [3]:
# Путь к объединенным CSV-файлам
combined_path = "combined_csvs"
combined_files = [f for f in os.listdir(combined_path) if f.endswith(".csv")]

# Сортировка файлов по числу пакетов (например, _250_)
combined_files.sort(key=lambda f: int(re.search(r"(\d+)_packets", f).group(1)))

# Словарь моделей с параметром verbose
models = {
    "Random Forest": RandomForestClassifier(random_state=42, verbose=1, n_jobs=-1),
    "Gradient Boosting": GradientBoostingClassifier(random_state=42, verbose=1),
    "SVM": LinearSVC(verbose=1, random_state=42),
    "k-NN": KNeighborsClassifier(n_neighbors=5, n_jobs=-1),
    "Logistic Regression": LogisticRegression(
        solver="saga",
        multi_class="ovr",
        class_weight="balanced",
        max_iter=1000,
        verbose=1,
        n_jobs=-1,
        random_state=42,
    ),
}

# Создание папки для промежуточных файлов
intermediate_path = "binary_class_models_results"
os.makedirs(intermediate_path, exist_ok=True)

# Итоговый список для всех результатов
all_results = []

# Проход по каждому CSV-файлу
for combined_file in tqdm(combined_files, desc="Обработка файлов"):
    df = pd.read_csv(os.path.join(combined_path, combined_file))
    X = df.drop(columns=["Label"])
    y = df["Label"]

    # Извлечение размера пакетов из имени файла
    packet_size = int(re.search(r"(\d+)_packets", combined_file).group(1))

    # Разделение данных и нормализация в отдельном блоке try
    try:
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.3, random_state=42, stratify=y
        )
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
    except ValueError as e:
        print(f"  ! Ошибка при подготовке данных для файла '{combined_file}': {e}")
        continue

    # Проход по каждой модели
    for model_name, model in models.items():
        results = {
            "Model": model_name,
            "File": combined_file,
            "Packet Size": packet_size,
        }

        print(f"  - Обучение модели {model_name}")
        try:
            # Обучение модели с выводом прогресса
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)

            # Расчёт метрик
            results["Accuracy"] = accuracy_score(y_test, y_pred)
            results["Macro F1 Score"] = f1_score(y_test, y_pred, average="macro")
            results["Micro F1 Score"] = f1_score(y_test, y_pred, average="micro")
            results["Weighted F1 Score"] = f1_score(y_test, y_pred, average="weighted")
            results["Macro Precision"] = precision_score(
                y_test, y_pred, average="macro"
            )
            results["Micro Precision"] = precision_score(
                y_test, y_pred, average="micro"
            )
            results["Cohen's Kappa"] = cohen_kappa_score(y_test, y_pred)

            # Отчёт по каждому классу
            class_report = classification_report(y_test, y_pred, output_dict=True)
            for label, metrics in class_report.items():
                if isinstance(metrics, dict):
                    results[f"Precision_{label}"] = metrics["precision"]
                    results[f"Recall_{label}"] = metrics["recall"]
                    results[f"F1_{label}"] = metrics["f1-score"]

        except ValueError as e:
            # Пропуск метрик при ошибке
            results.update(
                {
                    "Accuracy": "0.0",
                    "Macro F1 Score": "0.0",
                    "Micro F1 Score": "0.0",
                    "Weighted F1 Score": "0.0",
                    "Macro Precision": "0.0",
                    "Micro Precision": "0.0",
                    "Cohen's Kappa": "0.0",
                }
            )
            print(
                f"  ! Ошибка обучения модели '{model_name}' для файла '{combined_file}': {e}"
            )

        # Добавление результата
        all_results.append(results)

        # Промежуточный CSV для модели
        model_results_df = pd.DataFrame([results])
        model_results_file = os.path.join(
            intermediate_path, f"{model_name}_multiclass_results.csv"
        )

        # Если файл существует, добавляем данные
        if os.path.exists(model_results_file):
            model_results_df.to_csv(
                model_results_file, mode="a", index=False, header=False, float_format="%.3f"
            )
        else:
            model_results_df.to_csv(model_results_file, index=False, float_format="%.3f")

# Итоговый файл с результатами всех моделей
final_results_df = pd.DataFrame(all_results)
final_results_df.to_csv("final_multiclass_models_results.csv", index=False, float_format="%.3f")
print("Итоговый файл сохранён в 'final_multiclass_models_results.csv'")

Обработка файлов:   0%|          | 0/9 [00:00<?, ?it/s]

  - Обучение модели Random Forest


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 28 concurrent workers.
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   14.8s finished
[Parallel(n_jobs=28)]: Using backend ThreadingBackend with 28 concurrent workers.
[Parallel(n_jobs=28)]: Done 100 out of 100 | elapsed:    0.4s finished


  - Обучение модели Gradient Boosting
      Iter       Train Loss   Remaining Time 
         1           1.2126            1.38m
         2           1.0939            1.35m
         3           0.9959            1.33m
         4           0.9138            1.30m
         5           0.8444            1.28m
         6           0.7844            1.26m
         7           0.7331            1.23m
         8           0.6883            1.22m
         9           0.6494            1.20m
        10           0.6161            1.18m
        20           0.4371            1.05m
        30           0.3792           54.91s
        40           0.3514           46.90s
        50           0.3357           39.39s
        60           0.3242           31.44s
        70           0.3156           23.41s
        80           0.3095           15.58s
        90           0.3055            7.79s
       100           0.3018            0.00s
  - Обучение модели SVM
[LibLinear]  - Обучение модели k-NN
 

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 28 concurrent workers.


...

convergence after 555 epochs took 0 seconds
  - Обучение модели Random Forest


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 28 concurrent workers.
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=28)]: Using backend ThreadingBackend with 28 concurrent workers.
[Parallel(n_jobs=28)]: Done 100 out of 100 | elapsed:    0.0s finished


  - Обучение модели Gradient Boosting
      Iter       Train Loss   Remaining Time 
         1           1.1759            0.70s
         2           1.0289            0.49s
         3           0.9084            0.39s
         4           0.8071            0.36s
         5           0.7211            0.32s
         6           0.6448            0.31s
         7           0.5807            0.29s
         8           0.5228            0.29s
         9           0.4737            0.28s
        10           0.4308            0.27s
        20           0.1801            0.22s
        30           0.0792            0.19s
        40           0.0379            0.16s
        50           0.0210            0.14s
        60           0.0114            0.11s
        70           0.0069            0.08s
        80           0.0043            0.05s
        90           0.0027            0.03s
       100           0.0019            0.00s
  - Обучение модели SVM
[LibLinear]  - Обучение модели k-NN
 

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 28 concurrent workers.
Обработка файлов: 100%|██████████| 9/9 [07:10<00:00, 47.89s/it]

convergence after 425 epochs took 0 seconds
Итоговый файл сохранён в 'final_multiclass_models_results.csv'


In [4]:
# Создание папки для модифицированных промежуточных файлов
intermediate_path_modified = "binary_class_models_results_modified"
os.makedirs(intermediate_path_modified, exist_ok=True)

# Путь к промежуточным файлам
intermediate_path = "binary_class_models_results"
model_results_files = [f for f in os.listdir(intermediate_path) if f.endswith("_multiclass_results.csv")]

# Создаем итоговый DataFrame для всех моделей
final_results = []

# Процесс обработки каждого промежуточного файла модели
for model_results_file in model_results_files:
    # Чтение промежуточного CSV файла
    df = pd.read_csv(os.path.join(intermediate_path, model_results_file))
    # display(df)

    # Получаем список метрик, исключая "Packet Size" и "Model"
    metrics_columns = [col for col in df.columns if col not in ['Packet Size', 'Model', 'File']]

    # Определение лучшей выборки для каждой метрики
    best = {}
    counts = {}

    # Добавление столбца Counts для каждой выборки
    max1_counts_column = []
    for index, row in df.iterrows():
        max1_count = sum(1 for col in metrics_columns if row[col] == df[col].max())  # Считаем, сколько раз максимальное значение встречается
        max1_counts_column.append(max1_count)

    df["max 1"] = max1_counts_column

    # Для каждой метрики находим наилучшие выборки
    for col in metrics_columns:  # Перебираем только метрики
        max_value = df[col].max()  # Находим максимальное значение для метрики
        best[col] = df[df[col] == max_value]["Packet Size"].tolist()  # Сохраняем размеры выборок с максимальным значением

        # Подсчитаем, сколько раз каждая выборка была максимальной для этой метрики
        counts[col] = df[col].value_counts().get(max_value, 0)  # Подсчитаем сколько раз максимальное значение встречается

    # Строка Best
    best_row = {"Packet Size": "Best"}
    for col in metrics_columns:
        best_row[col] = best[col]  # Присваиваем лучшие выборки для каждой метрики

    # Добавление строки Best в DataFrame
    df_best = pd.DataFrame([best_row])

    # Добавление строки Best в DataFrame
    df = pd.concat([df, df_best], ignore_index=True)
    
    model_results_file_modified = os.path.join(
            intermediate_path_modified, f"{os.path.splitext(model_results_file)[0]}_modified.csv"
        )

    # Запись результатов в новый CSV файл
    df.to_csv(model_results_file_modified, index=False)